In [ ]:
from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import math
import ROOT
import numpy as np, pandas as pd
import yaml
import base64
import ctypes
import logging
import bokeh
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import bokehDrawArray
from RootInteractive.Tools.pandaTools import initMetadata
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.Tools.compressArray import arrayCompressionRelative8

output_notebook()

In [ ]:
#inputFile="/lustre/alice/users/miranov/NOTESData/alice-tpc-notes/JIRA/ATO-592/tpcqcskimmingTracks.root"
inputFile="~/Software/TPCQCVis/data/tpcqcskimmingTracks.root"

In [ ]:
f=ROOT.TFile.Open(inputFile)
tracks = f.Get("tracks;2")  
tracks.SetAlias("weight1","((triggerMask&0x1)>0)*max(weight,0.01)")
tracks.SetAlias("weight2","((triggerMask&0x2)>0)*max(weight*pt*pt,0.01)")
tracks.SetAlias("rMin","trackPar.mX")
tracks.SetAlias("qPt","trackPar.mP[4]")
tracks.SetAlias("pzPt","trackPar.mP[3]")
tracks.SetAlias("itsOn","itsClusterMap>0")
tracks.SetAlias("tofOn","abs(tofChi2)<100")
tracks.SetAlias("p","pt*sqrt(1+trackPar.mP[3])")
tracks.SetAlias("logp","log(p)")
tracks.SetAlias("logtpcSignal","log(tpcSignal)")

In [ ]:
df=tree2Panda(tracks, [".*"], "", exclude=["trackPar"])   
df["dcaDefined"]=df.eval("dcaXY!=0")
df["side"]=df.eval("pzPt>0") ##  (A/C/CrossAC/CrossCA side)
df["isPrim5"]=df.eval("abs(dcaXY)<5 &  abs(dcaZ)<5 & rMin<5")
df["tpcNCR"]=df["tpcNClsFindable"]-df["tpcNClsFindableMinusCrossedRows"]

In [ ]:
print(df.columns)

In [ ]:
aliasArray = []

In [ ]:
parameterArray = [
    {"name": "size", "value":5, "range":[0, 20]},
    {"name": "legendFontSize", "value":"13px", "options":["9px", "11px", "13px", "15px"]},
    {"name": "binCountTrans", "value":"bin_count", "options":["bin_count", "logbin_count","sqrtbin_count"]},
    {"name": "legendVisible", "value":True},
    {"name": "legendLocation", "value":"top_right", "options":["top_right","top_left", "bottom_right","bottom_left"]},
]

In [ ]:
widgetParams=[
      ['range', ['pt']],
      ['range', ['qPt']],
      ['range', ['weight']],
      ['range', ['weight1']],
      ['range', ['weight2']],
      ['multiSelect',["triggerMask"]],
      ['textQuery', {"title": "user select"}],
      # Graphics
      ['slider',["size"], {"callback": "parameter", "title":"Marker size"}],
      ['select',["binCountTrans"], {"callback": "parameter", "default": 0, "title":"Bin count transformation"}],
      ['toggle',['legendVisible']],  
      ['select',["legendFontSize"], {"callback": "parameter", "default": 2, "title":"Legend font size"}],
      ['select',["legendLocation"], {"callback": "parameter", "default": 0}]
]
widgetLayoutDesc={
    "Selection:": [[0,1],[2,3,4],[5,6], {'sizing_mode': 'scale_width'}],
    "Graphics": [[7,8,9,10], {'sizing_mode': 'scale_width'}]
}

In [ ]:
histoArray = [
    {"name": "pthist", "variables": ["pt"], "nbins":100},
    {"name": "qPthist", "variables": ["qPt"], "nbins":100},
    {"name": "weighthist", "variables": ["weight"], "nbins":500},
    {"name": "weight1hist", "variables": ["weight1"], "nbins":500},
    {"name": "weight2hist", "variables": ["weight2"], "nbins":500},
    {"name": "weightVsPt", "variables": ["pt","weight"], "nbins":[100,100]},
    {"name": "weight1VsPt", "variables": ["pt","weight1"], "nbins":[100,100]},
    {"name": "weight2VsPt", "variables": ["pt","weight2"], "nbins":[100,100]},
    {"name": "weight2VsqPt", "variables": ["qPt","weight2"], "nbins":[100,100]}
]

In [ ]:
histoArrayLog=[]
for his in histoArray:
    histoName=his["name"]    
    histoArrayLog.append({ "name": f"log{histoName}","variables": ["bin_count"],"func": "return Math.log(bin_count+0.2)","context": f"{histoName}"})
    histoArrayLog.append({ "name": f"log{histoName}Err","variables": ["bin_count"],"func": "return 1/Math.sqrt(bin_count+0.2)","context": f"{histoName}"})               
    histoArrayLog.append({ "name": f"{histoName}Err","variables": ["entries","std"],"func": "return std/Math.sqrt(entries+0.1)","context": f"{histoName}_1"})
    #
    histoArrayLog.append({ "name": f"logbin_count","variables": ["bin_count"],"func": "return Math.log(bin_count+0.2)","context": f"{histoName}"})
    histoArrayLog.append({ "name": f"sqrtbin_count","variables": ["bin_count"],"func": "return Math.sqrt(bin_count+0.2)","context": f"{histoName}"})
aliasArray.extend(histoArrayLog)

In [ ]:
tooltips = []

In [ ]:
figureArray = [
    [['bin_center'],['binCountTrans'],{'source':"pthist",'size': 5,"xAxisTitle":"pT","yAxisTitle":"counts"}],
    [['bin_center'],['binCountTrans'],{'source':"qPthist",'size': 5,"xAxisTitle":"qPt","yAxisTitle":"counts"}],
    [['bin_center'],['binCountTrans'],{'source':"weighthist",'size': 5,"xAxisTitle":"weight","yAxisTitle":"counts"}],
    [['bin_center'],['binCountTrans'],{'source':"weight1hist",'size': 5,"xAxisTitle":"weight1","yAxisTitle":"counts"}],
    [['bin_center'],['binCountTrans'],{'source':"weight2hist",'size': 5,"xAxisTitle":"weight2","yAxisTitle":"counts"}],
    [['pt'],['weightVsPt'],{"xAxisTitle":"pT","yAxisTitle":"weight"}],
    [['pt'],['weight2VsPt'],{"xAxisTitle":"pT","yAxisTitle":"weight2"}],
    [['qPt'],['weight2VsqPt'],{"xAxisTitle":"pT","yAxisTitle":"weight2"}],
    {'plot_height':200, "commonX":0, "size":"size",
     "legend_options": {"label_text_font_size": "legendFontSize", "visible": "legendVisible", "location":"legendLocation"}}
]            

In [ ]:
figureLayout = [
    [[0,1],[2,3,4],[5,6,7], {'plot_height':300,'sizing_mode':'scale_width',"legend_visible":True}]
]

In [ ]:
output_file("track_skimming.html")
xxx = bokehDrawSA.fromArray(df, "pt<20&abs(qPt)<20&abs(pzPt)<2", figureArray, widgetParams, layout=figureLayout, tooltips=tooltips, widgetLayout=widgetLayoutDesc,
                            parameterArray=parameterArray, histogramArray=histoArray,aliasArray=aliasArray,sizing_mode='scale_width',
                            arrayCompression=arrayCompressionRelative8)